In [1]:
from importnb import Notebook 
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import struct

In [2]:
import numpy as np
import math

class layer:
    def __init__(self,Input_len,nodenum,acti_name):
        self.nodenum = nodenum
        # temp =  np.full((Input_len+1,nodenum),1)
        # self.Input_w = np.random.random( (Input_len+1,nodenum) )*2-temp
        # self.Input_w = np.random.random(Input_len+1,nodenum)
#         self.Input_w = np.full((Input_len+1,nodenum),0.5)
        self.Input_w = np.random.randn(Input_len+1,nodenum)/np.sqrt(Input_len+1)
        if acti_name == 'relu':
            self.actiFunc = self.relu
            self.actiFunc_diff = self.drelu
        elif acti_name == 'softmax':
            self.actiFunc = self.softmax
            self.actiFunc_diff = self.dsoftmax   # 隨便給
        elif acti_name == 'sigmoid':
            self.actiFunc = self.sigmoid
            self.actiFunc_diff = self.dsigmoid
            
    def give_input(self,Input):
        self.Input = Input.copy()
        
    def cal_output(self):
        # 算出系統output
        self.weighted_Input = np.dot(self.Input, self.Input_w)
        self.Output = self.actiFunc(self.weighted_Input)
#         print("in: ", self.Input, "w: ", self.Input_w, "out: ", self.Output)
        return self.Output
    
    """ Activation function & Differential function """  
    def relu(self,x_value):
        x_value[x_value<=0] = 0
        return x_value.copy()
    
    def drelu(self,x_value):
        x_value[x_value > 0] = 1
        x_value[x_value < 0] = 0
        return x_value.copy()
            
    def softmax(self,x_value):
        x_value = x_value-np.max(x_value)
        return np.exp(x_value)/ np.sum(np.exp(x_value))
    
    def dsoftmax(self,x_value):
        return 1

    def sigmoid(self,x_value):
        output = 1/(1+np.exp(-x_value))
        return output
        
    def dsigmoid(self,x_value):    
        return np.exp(-x_value)/np.square( (1+np.exp(-x_value)) )
        
#         self.Input_w = np.full((Input_len+1,nodenum),0.5)
#         self.Input_w =  np.random.random( (Input_len+1,nodenum) ) # 加一是因為把bias含進去weight裡面
    

In [3]:
class model:
    def __init__(self,loss_name,learn_rate,data_list,input_len):
        self.Layer_list = list() # 把 layer 按照先後順序放入
        self.input_len = input_len   # 要給輸入長度才有辦法使用 model_input
        self.learn_rate = learn_rate
        self.model_input = data_list  # 輸入加答案(方便洗牌)
        self.standard = list()
        
        if loss_name == 'MSE':
            self.loss_diff = self.mse_diff
        elif loss_name == 'cross_entropy':
            self.loss_diff = self.dsoftmax_entropy
    
    # train model        
    def model_train(self,Iter,epoch):
        for i in range(Iter):
            # np.random.shuffle(self.model_input)
            correct=list()
            loss = list()
            for j in range(int(len(self.model_input)/epoch)):
                
                
                temp_input = self.model_input[j*epoch:(j+1)*epoch]
                bp_input = np.zeros(self.Layer_list[len(self.Layer_list)-1].nodenum)
                output = list()
                ans = list()
                for item in temp_input:
#                     print(item, "s, ", item[0:self.input_len],item[self.input_len:len(item)])
                    temp_op,temp_bpip = self.forward_pass(item[0:self.input_len],item[self.input_len:len(item)])
                    output.append(temp_op.copy())
                    ans.append(item[self.input_len:len(item)])
                    bp_input += temp_bpip.copy()
#                     for k in range(len(temp_bpip)):
#                         bp_input[k] += temp_bpip[k]
                    loss.append( - np.sum(item[self.input_len:len(item)] * np.log(temp_op)))
                    
                    # print("output",output)
                self.backpropagation(bp_input/epoch)
                correct.append(self.dealoutput(output,ans))
            print("第",i,np.average(correct), "loss: ", np.average(loss))
    
    def model_test(self,testdata_list):
        for item in testdata_list:
            temp_op,temp_bpip = self.forward_pass(item[0:self.input_len],item[self.input_len:len(item)])  
            output.append(temp_op)
            ans.append(item[len(item)-1])
        correct = self.dealoutput(output,ans)
        print("正確率",correct)
        
    # 加入 layer 
    def setlayer(self,List):  
        self.Layer_list = List.copy()
        
    """ loss function """ 
    def mse_diff(self,output,standard):
#         print(output-standard)
        return output-standard
    
    def dsoftmax_entropy(self,output,standard):
#         print("output: ",output, "std: ", standard, "diff: ", output-standard)
        return output-standard
    
    def forward_pass(self,Input,standard):
        # 進行一次 input forward pass運算
        Input = np.append(Input.copy(),1)
        self.Layer_list[0].give_input(Input) 
#         print(self.Layer_list)
        for i in range(len(self.Layer_list)):
            # 前一層output當做下一層的輸入 
            output = self.Layer_list[i].cal_output()
            if i+1< len(self.Layer_list):
                output = np.append(output,1)
                self.Layer_list[i+1].give_input(output)
        bp_input = self.loss_diff(output,standard)
        return output,bp_input  # 最終output

#     def backpropagation(self,bp_input):
#         layer_reverse = self.Layer_list.copy()
#         layer_reverse.reverse()
#         for layer in layer_reverse:
#             # print('before in: ',bp_input,"df: ", layer.actiFunc_diff(layer.weighted_Input))
#             bp_error = bp_input*layer.actiFunc_diff(layer.weighted_Input)
#             # print(self.learn_rate,bp_error,layer)
#             Input_w = layer.Input_w.transpose()
#             bp_input = bp_error.dot(Input_w[:,0:len(Input_w[0])-1])
# #             print('before',layer.Input_w)
#             for i in range(len(layer.Input)):
#                 # print("in ", i, self.learn_rate,bp_error,layer.Input[i])
#                 layer.Input_w[i] = layer.Input_w[i] - self.learn_rate*bp_error*layer.Input[i]
#                 # print("第",i,layer.Input_w[i])
# #             print('after',layer.Input_w)
#         # self.Layer_list = layer_reverse.reverse()
    
    def backpropagation(self,bp_input):
        for j in range(len(self.Layer_list)-1, -1, -1):
            layer = self.Layer_list[j]
            delta = bp_input * layer.actiFunc_diff(layer.weighted_Input)    # Dimation of layer node
            delta_w = np.outer(layer.Input, delta)
            bp_input = np.dot(delta, layer.Input_w[0:len(layer.Input_w)-1, :].transpose())
            layer.Input_w -= self.learn_rate*delta_w
    
    def dealoutput(self,output,standard):
        result = list()
        for i in range(len(output)):
            ans = 1 if np.argmax(output[i]) == np.argmax(standard[i]) else 0 
            result.append(ans)
#         print("print",result.count(1),len(result))
        
        correct = np.average(result)
#         for i in range(len(output)):
#             print("output: ",output[i], "std: ", standard[i], "result: ", result[i], "correct: ", correct)
#         print("correct: ", correct)
        
#         while True:
#             pass
        return correct

In [4]:
def read_data(image_dir, label_dir):
    with open(image_dir, 'rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        
        buf = file.read(size * rows * cols)
        img_data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        img_data = img_data.reshape(size, rows*cols)
        img_data = img_data / 255
        
    
    with open(label_dir, 'rb') as file:
        magic, size = struct.unpack(">II", file.read(8))
        
        buf = file.read(size)
        lab_data_tmp = np.frombuffer(buf, dtype=np.uint8).astype(np.uint16)
        lab_data_tmp = lab_data_tmp.reshape(size)
        
#     print(lab_data[0:111])
    lab_data = np.zeros((size, 10), dtype=np.uint16)
    for i in range(size):
        lab_data[i][ lab_data_tmp[i] ] = 1
    
#     print(lab_data[0:10])
    
    
    return img_data.copy(), lab_data.copy()

In [5]:
# print(np.frombuffer(b'\x02\x01\x00\x00', dtype=np.uint16))
X_train, Y_train  = read_data('code/MNIST/train-images-idx3-ubyte', 'code/MNIST/train-labels-idx1-ubyte')
X_test, Y_test  = read_data('code/MNIST/t10k-images-idx3-ubyte', 'code/MNIST/t10k-labels-idx1-ubyte')

In [6]:
"""處理輸入資料"""

# Load MNIST data
train_data = np.append(X_train,Y_train, axis=1)
# 合併資料跟答案
# for i in range(len(X_train)):
#     train_data.append(X_train[i],Y_train[i])

# print(train_data[1])


In [7]:
"""進行訓練"""
test_input = train_data
test_model = model("cross_entropy",0.0047,test_input,784) # loss_name,learn_rate,model_input,input_len

# layer1 = layer(14,65,'sigmoid') #Input_len,nodenum,acti_name
# layer2 = layer(65,32,'relu') #Input_len,nodenum,acti_name
# layer3 = layer(32,1,'sigmoid') #Input_len,nodenum,acti_name

layer1 = layer(784,50,'relu') #Input_len,nodenum,acti_name
layer2 = layer(50,23,'relu') #Input_len,nodenum,acti_name
layer3 = layer(23,17,'relu') #Input_len,nodenum,acti_name
layer4 = layer(17,10,'softmax') #Input_len,nodenum,acti_name

layer_list = [layer1,layer2,layer3,layer4]
test_model.setlayer(layer_list)
test_model.model_train(5000,1)

第 0 0.9042666666666667 loss:  0.316989316683963
第 1 0.9572 loss:  0.1440500488037263
第 2 0.9660833333333333 loss:  0.11306652652206726
第 3 0.9725333333333334 loss:  0.09277404722163868
第 4 0.97605 loss:  0.08064843650671025
第 5 0.9781333333333333 loss:  0.07204221415269849
第 6 0.9799833333333333 loss:  0.06582101031220143
第 7 0.9816666666666667 loss:  0.05876553826248052
第 8 0.9821 loss:  0.05665563586138602
第 9 0.98375 loss:  0.05174067997683998
第 10 0.9843333333333333 loss:  0.049121671489253775
第 11 0.98545 loss:  0.04632806750421893
第 12 0.9867 loss:  0.043570346663668644
第 13 0.9874 loss:  0.04068261734073044
第 14 0.9877166666666667 loss:  0.039416989683056666
第 15 0.9876 loss:  0.03863257849991415
第 16 0.9883333333333333 loss:  0.03652969450807379
第 17 0.9890166666666667 loss:  0.03410382174364254
第 18 0.9896166666666667 loss:  0.03182942072164167
第 19 0.9888 loss:  0.03540446467399903
第 20 0.9901 loss:  0.03138166758875857
第 21 0.9896333333333334 loss:  0.03258999434660195
第 22 

KeyboardInterrupt: 